In [1]:
# Imports
import pandas as pd
import os
import typing
from functools import partial
from pprint import pprint
from pypif import pif

In [2]:
# Load Data
# Filter to comp
# Parse Vector
# Upload to citrination

In [3]:
load_path = os.path.join('data','training_data.csv')
save_path = os.path.join('data','processed_data.csv')

In [4]:
df = pd.read_csv(load_path)

In [5]:
original_cols = df.columns

In [6]:
cols = ['{}_percent'.format(i*10) for i in range(11)]

In [7]:
def debug(func):
    def debug_wrapper(*args, **kwargs):
        row = args[0]
        print(row['stabilityVec'])
        func(*args, **kwargs)
    return debug_wrapper

In [8]:
def vec_to_stability(row: pd.Series, cols: list) -> pd.Series:

    vec = eval(row['stabilityVec'])
    for element, col in zip(vec, cols):
        row[col] = int(element)
    
    return row

In [9]:
vtf = partial(vec_to_stability, cols=cols)

In [10]:
df = df.apply(vtf, axis=1)

In [11]:
df[['formulaA', 'formulaB']+cols].to_csv(save_path, index=False)

In [12]:
df.head()

,formulaA,formulaB,formulaA_elements_AtomicVolume,formulaB_elements_AtomicVolume,formulaA_elements_AtomicWeight,formulaB_elements_AtomicWeight,formulaA_elements_BoilingT,formulaB_elements_BoilingT,formulaA_elements_BulkModulus,formulaB_elements_BulkModulus,...,10_percent,20_percent,30_percent,40_percent,50_percent,60_percent,70_percent,80_percent,90_percent,100_percent
0,Ne,He,37.232186,37.236036,20.179100,4.002602,26.92,4.07,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,Cs,He,117.456016,37.236036,132.905452,4.002602,944.00,4.07,1.6,0.0,...,0,0,0,0,0,0,0,0,0,1
2,K,He,75.847865,37.236036,39.098300,4.002602,1032.00,4.07,3.1,0.0,...,0,0,0,0,0,0,0,0,0,1
3,Ba,He,64.969282,37.236036,137.327000,4.002602,2143.00,4.07,9.6,0.0,...,0,0,0,0,0,0,0,0,0,1
4,Sr,He,55.323131,37.236036,87.620000,4.002602,1655.00,4.07,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1


## Pivot each row and convert the stability vector into a chemical formula 
Pivot stability vector

In [13]:
binary_df = df.melt(id_vars=original_cols, var_name='weight_fraction_element_b', value_name='stable')

In [14]:
binary_df['weight_fraction_element_b'] = binary_df['weight_fraction_element_b'].str.strip('_percent').astype(float)

In [17]:
binary_df['weight_fraction_element_b'] = binary_df['weight_fraction_element_b']/100

Convert `weight_percent` to formula

In [34]:
def row_to_formula(row: pd.Series):
    w = row['weight_fraction_element_b']
    a = row['formulaA']
    b = row['formulaB']
    
    if w == 0:
        return a
    elif w == 100:
        return b
    else:
        wa = 1.0-w
        def compute_formula(wa):
            return f"{a}{wa:.1f} {b}{w:.1f}"
        return compute_formula(wa)
             

In [35]:
binary_df['formula'] = binary_df.apply(row_to_formula, axis=1)

Export data

In [43]:
# only export unique
binary_df = binary_df.drop_duplicates('formula')

In [44]:
binary_df[['formula','formulaA','formulaB','stable']].to_csv(save_path, index=False)